In [ ]:
import gymnasium as gym
from pystk2_gymnasium import AgentSpec
from stk_actor.training_strategies.expert_trajectories import parallel_run_episodes

tracks = [
    'abyss',
    'black_forest',
    'candela_city',
    'cocoa_temple',
    'cornfield_crossing',
    'fortmagma',
    'gran_paradiso_island',
    'lighthouse',
    'mines',
    'minigolf',
    'olivermath',
    'hacienda',
    'ravenbridge_mansion',
    'sandtrack',
    'scotland',
    'snowmountain',
    'snowtuxpeak',
    'stk_enterprise',
    'volcano_island',
    'xr591',
    'zengarden',
    # HARD
    # 'fortmagma',
    # 'ravenbridge_mansion',
    # 'snowmountain',
    # 'cocoa_temple',
    # 'sandtrack',    
    # 'scotland', 
    # 'stk_enterprise',
    # 'volcano_island', # 1104
    # 'xr591', # 864
    # 'hacienda',
    # "snowtuxpeak",
]

args = []
nb_runs = 4
for lap in [3]:
    for num_karts in [2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
        for difficulty in [2]:
            for track in tracks:
                for _ in range(nb_runs):
                    args.append((track, difficulty, lap, num_karts))

# ipcluster start -n 16
records = parallel_run_episodes(args)
print(len(records))

In [ ]:
from stk_actor.replay_buffer import create_and_fill_replay_buffer
from pystk2_gymnasium.stk_wrappers import ConstantSizedObservations, PolarObservations, DiscreteActionsWrapper
from pystk2_gymnasium.wrappers import FlattenerWrapper

env = gym.make(
    "supertuxkart/full-v0",
    render_mode=None,
    agent=AgentSpec(use_ai=False, name="walid"),
    track='abyss',
    num_kart=2,
    difficulty=0
)

env = FlattenerWrapper(
        DiscreteActionsWrapper(
            PolarObservations(
                ConstantSizedObservations(
                    env,
                    state_items = 5,
                    state_karts = 5, 
                    state_paths = 5,
                )
            )
        )
)

buffer = create_and_fill_replay_buffer(env, records)

env.close()


In [ ]:
import joblib
joblib.dump(buffer,'all_tracks_buffer_expl', compress=4)